<a href="https://colab.research.google.com/github/Elwing-Chou/ml0223/blob/main/transfer_real.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rarfile

In [ ]:
import rarfile
path = "./水獺.rar"
f = rarfile.RarFile(path)
f.extractall()

In [ ]:
import glob
from PIL import Image
fs = glob.glob("水獺/*")
Image.open(fs[0]).resize((224, 224)).convert("RGB")

In [ ]:
import rarfile
path = "./海狸.rar"
f = rarfile.RarFile(path)
f.extractall()

In [ ]:
import glob
from PIL import Image
fs = glob.glob("海狸/*")
Image.open(fs[0]).resize((224, 224)).convert("RGB")

In [ ]:
import pandas as pd
dog = glob.glob("海狸/*")
cat = glob.glob("水獺/*")
datas = pd.DataFrame({
    "path":dog + cat,
    "ans":[0] * len(dog) + [1] * len(cat)
})
datas

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
vgg = VGG16(include_top=False, input_shape=(224, 224, 3))
vgg.summary()

In [ ]:
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
# Trainbale + 1024 (Step2: 最優化)
# Non + 1024 (Step1: 標準標準化)
vgg_layers = vgg.layers
for l in vgg_layers:
    l.trainable = False
clf_layers = [
    BatchNormalization(),
    GlobalAveragePooling2D(),
    Dense(2, activation="softmax")
]
model = Sequential(vgg_layers + clf_layers)
model.summary()

In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
model.compile(loss=SparseCategoricalCrossentropy(), 
       optimizer="adam",
       metrics=["accuracy"])

In [ ]:
from sklearn.model_selection import train_test_split
x, y = np.array(datas["path"]), np.array(datas["ans"])
x, x_test, y, y_test = train_test_split(x, y, test_size=0.05)

In [ ]:
print(x.shape)
print(x_test.shape)
print(y.shape)
print(y_test.shape)

In [ ]:
def get_data(x, y, batch=20):
    idx = np.random.randint(0, len(x), size=batch)
    y_batch = []
    x_batch, x_batch_ori = [], []
    for i in idx:
        try:
            p = x[i]
            img = Image.open(p).resize((224, 224)).convert("RGB")
            img_np = np.array(img) 
            img_np_pre = preprocess_input(img_np)
            x_batch.append(img_np_pre)
            x_batch_ori.append(img_np)
            y_batch.append(y[i])
        except:
            pass
    return (np.array(x_batch), np.array(y_batch), np.array(x_batch_ori))
x_batch, y_batch, x_batch_ori = get_data(x, y)
print(x_batch.shape)
print(y_batch.shape)
print(x_batch_ori.shape)

In [ ]:
for i in range(20):
    print("-" * 15, i, "-" * 15)
    x_batch, y_batch, x_batch_ori = get_data(x, y)
    result = model.train_on_batch(x_batch, y_batch)
    print("[Train]:", result)
    x_batch, y_batch, x_batch_ori = get_data(x, y)
    result = model.test_on_batch(x_batch, y_batch)
    print("[Validate]:", result)

In [ ]:
x_batch, y_batch, x_batch_ori = get_data(x_test, y_test, 50)
model.evaluate(x_batch, y_batch)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

pre = model.predict_classes(x_batch)
trans = ["sea", "water"]
idx = np.nonzero(pre != y_batch)[0][:200]
false_img = x_batch_ori[idx]
false_pre = pre[idx]
false_label = y_batch[idx]
plt.figure(figsize=(14, 14))
width = 10
height = len(idx) // width + 1
for i in range(len(idx)):
    plt.subplot(height, width, i+1)
    t = "[O]:{}\n[P]:{}".format(trans[false_label[i]], trans[false_pre[i]])
    plt.title(t)
    plt.axis("off")
    plt.imshow(false_img[i])

In [ ]:
# Pillow
import requests
from PIL import Image
# RGB, RGBA, L(gray)
url = input("url:")
headers = {
  "user-agent":"user-agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36"  
}
response = requests.get(url, headers=headers, stream=True, verify=False)
img = Image.open(response.raw).resize((224, 224)).convert("RGB")
img_np = np.array(img)
img_np_r = preprocess_input(img_np.reshape(1, 224, 224, 3))
proba = model.predict(img_np_r)[0]
for i, p in enumerate(proba):
    print(trans[i], "的機率:", p)
ans = model.predict_classes(img_np_r)[0]
print("答案:", trans[ans])
plt.imshow(img_np)